Paso 1: Importar librerías necesarias

In [121]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import ast

Paso 2: Cargar y explorar el dataset

In [122]:
movies_df = pd.read_csv('movies_dataset.csv')

C:\Users\Frank y Cami\AppData\Local\Temp\ipykernel_22480\3533197153.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_df = pd.read_csv('movies_dataset.csv')


2.1 Se observan las dimensiones del dataset

In [123]:
movies_df.shape

(45466, 24)

se validan inicialmente 45466 filas y 24 columnas

2.2 Se observan las variables categoricas y numericas

In [124]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

Se observan inicialmente columnas 4 numericas y se desanidan las claves mas reelevantes en cada columna

In [125]:
# Extraer nombres de géneros
movies_df['genres'] = movies_df['genres'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) else[])

In [126]:
# Extraer nombres de lenguaje
movies_df['spoken_languages'] = movies_df['spoken_languages'].apply(lambda x: [i['iso_639_1'] for i in ast.literal_eval(x)] if pd.notnull(x) else [])

In [127]:
# responder si pertenece a una coleccion
movies_df['belongs_to_collection'] = movies_df['belongs_to_collection'].apply(lambda x: 1 if pd.notnull(x) else 0)

Paso 3 Se rellenan los valores nulos de los campos revenue y budget por el numero 0, se observa que la columna butget no tiene valores nulos mientras que revenue tiene 5 

In [128]:
movies_df['budget'].fillna(0,inplace=True)
movies_df['revenue'].fillna(0,inplace=True)

C:\Users\Frank y Cami\AppData\Local\Temp\ipykernel_22480\2779611814.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  movies_df['budget'].fillna(0,inplace=True)
C:\Users\Frank y Cami\AppData\Local\Temp\ipykernel_22480\2779611814.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.


ahora se valida que las columnas revenue y budget no tienen valores nulos

Paso 4 se eliminan los valores nulos de release_date, esto eliminara 87 filas del dataset

In [129]:
movies_df.dropna(subset=['release_date'],inplace=True)

elimimar las columnas incecesarias

In [130]:
movies_df.drop(['video','imdb_id','adult','original_title','poster_path','homepage'],axis=1,inplace=True)

convertir el ano de la columna release date en formato

In [131]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

crear una columna release year

In [132]:
movies_df['release_date'] = pd.to_datetime(movies_df['release_date'], errors='coerce')
movies_df['release_year'] = movies_df['release_date'].dt.year

# Mostrar las primeras filas para verificar
movies_df[['release_date', 'release_year']].head()

,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0


In [133]:
movies_df[['release_date', 'release_year']].head()

,release_date,release_year
0,1995-10-30,1995.0
1,1995-12-15,1995.0
2,1995-12-22,1995.0
3,1995-12-22,1995.0
4,1995-02-10,1995.0


In [134]:
movies_df['release_date'].info()

<class 'pandas.core.series.Series'>
Index: 45379 entries, 0 to 45465
Series name: release_date
Non-Null Count  Dtype         
--------------  -----         
45376 non-null  datetime64[ns]
dtypes: datetime64[ns](1)
memory usage: 709.0 KB


este proceso genera 3 filas nulas en release date, por lo que se ejecuta el codigo nuevamente para eliminar estas filas

In [135]:
movies_df.dropna(subset=['release_date'],inplace=True)

In [136]:
# Convertir las columnas 'revenue' y 'budget' a tipo numérico en caso de que haya datos faltantes o no numéricos
movies_df['revenue'] = pd.to_numeric(movies_df['revenue'], errors='coerce')
movies_df['budget'] = pd.to_numeric(movies_df['budget'], errors='coerce')



In [137]:
# Calcular el retorno de inversión
movies_df['return'] = movies_df.apply(lambda x: x['revenue'] / x['budget'] if x['budget'] > 0 else 0, axis=1)


In [138]:
# Extraer nombres de companias de produccion
movies_df['production_companies'] = movies_df['production_companies'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)] if isinstance(x, str) and pd.notnull(x) else [])

In [139]:
# Extraer nombres de pais de produccion
movies_df['production_countries'] = movies_df['production_countries'].apply(lambda x: [i['name'] for i in ast.literal_eval(x)] if pd.notnull(x) else[])

In [140]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 45376 entries, 0 to 45465
Data columns (total 20 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  45376 non-null  int64         
 1   budget                 45376 non-null  int64         
 2   genres                 45376 non-null  object        
 3   id                     45376 non-null  object        
 4   original_language      45365 non-null  object        
 5   overview               44435 non-null  object        
 6   popularity             45376 non-null  object        
 7   production_companies   45376 non-null  object        
 8   production_countries   45376 non-null  object        
 9   release_date           45376 non-null  datetime64[ns]
 10  revenue                45376 non-null  float64       
 11  runtime                45130 non-null  float64       
 12  spoken_languages       45376 non-null  object        
 13  status

Nuestro dataset ahora es de 45465 filas y 20 columnas de clas cuales una tiene formato de fecha, 8 son numericas y 11 categoricas

In [141]:
movies_df.to_csv('archivotransformado.csv',index=False)

Pasamos a la limpieza del archivo credits.csv 

cargamos el archivo 

In [ ]:
data2 = pd.read_csv('credits.csv')
print(data2)

se valida que hay aninados en ambas columnas

In [ ]:
data2.info()

Proceso de desanidacion

In [ ]:
# Convertir la columna "cast" de string a una lista de diccionarios, si está en formato JSON-string
data2['cast'] = data2['cast'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Extraer los nombres en una nueva columna
data2['cast_names'] = data2['cast'].apply(lambda cast_list: [member['name'] for member in cast_list] if isinstance(cast_list, list) else [])

# Visualizar el resultado
print(data2[['cast', 'cast_names']].head())

Separaciond de columnas para facilitar el proceso

In [ ]:
# Crear un nuevo DataFrame solo con las columnas 'id' y 'cast_names'
actor_df = data2[['id', 'cast_names']]

# Guardar el nuevo DataFrame en un archivo CSV
actor_df.to_csv('actor.csv', index=False)

# Verificar el nuevo dataset
print(actor_df.head())

en este proceso procedere a separar los nombres de actores en filas individuales 

In [ ]:
# Separar los nombres de 'cast_names' en filas individuales
df_exploded = actor_df.assign(cast_names=actor_df['cast_names'].astype(str).str.split(',')).explode('cast_names')

# Limpiar espacios en blanco de los nombres
df_exploded['cast_names'] = df_exploded['cast_names'].str.strip()

# Eliminar caracteres no deseados, dejando solo letras y espacios
df_exploded['cast_names'] = df_exploded['cast_names'].str.replace(r"[^a-zA-Z\s]", "", regex=True)

# Resetear el índice para eliminar los índices antiguos
df_exploded = df_exploded.reset_index(drop=True)

# Mostrar el resultado
print(df_exploded)

In [ ]:
df_exploded.to_csv('final.csv')

En este punto se procede a trabajar ahora con la extraccion de los directores, ya que se encuentran en celdas anidadas

In [ ]:
# Función para extraer el nombre del director
def obtener_director(crew_data):
    # Convertir el texto JSON a una lista de diccionarios
    crew_list = ast.literal_eval(crew_data)
    # Filtrar para encontrar el director
    for miembro in crew_list:
        if miembro.get('job') == 'Director':
            return miembro.get('name')
    return None

# Aplicar la función a la columna 'crew' y crear una nueva columna 'director'
data2['director'] = data2['crew'].apply(obtener_director)

# Mostrar las primeras filas para verificar
data2[['crew', 'director']].head()

Se crea un dataframe que se usara temporalmente con director y id

In [ ]:
# Crear un nuevo DataFrame solo con las columnas 'id' y 'director'
df_directors = data2[['id', 'director']]

# Guardar el nuevo DataFrame en un archivo CSV
df_directors.to_csv('director.csv', index=False)

se procede a unir estos 2 archivos para tener el dataset desanidado de id, actor y director

In [ ]:
# Realizar el merge de ambos dataframes a través de la columna 'id'
merged_df = pd.merge(df_exploded, df_directors, on='id', how='outer')
print(merged_df)

In [ ]:
merged_df.to_csv('merged.csv')

los archivos previamente guardados como csv procederemos a cargarlos como dataframes, llamaremos principal al de movies y secundario al de credits

In [ ]:
df_principal= pd.read_csv('archivotransformado.csv')
df_secundaria = pd.read_csv('merged.csv',index_col=0)

Eliminar duplicados antes del merge:

Si alguno de los dataframes tiene filas duplicadas en la columna id, puedes eliminarlos antes de hacer el merge. Esto ayudará a evitar combinaciones innecesarias que aumentan el tamaño del archivo.

In [ ]:
df_secundaria = df_secundaria.drop_duplicates(subset='id')

ahora procederemos a hacer la union de ambos datasets, tal como una funcion de Mysql usaremeos el parametro "outer" para que se tomen todas las columnas de ambos datasetS

In [ ]:
data_completa = pd.merge(df_principal, df_secundaria, on='id', how='outer')

Y asi tengo mi archivo csv el cual sera objeto de consulta en las APIS 

In [ ]:
data_completa.to_csv('archivov4.csv')